In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
os.mkdir('vit')

In [3]:
import torch
use_cuda = torch.cuda.is_available()

In [6]:
model_name = 'vit_go_orig-25e'

In [7]:
import os
import json
import zipfile
import random
import shutil
import time

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as kr

from matplotlib import pyplot as plt 
from PIL import Image
from keras import backend as K
from tensorflow.keras import layers, models
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']

caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']

  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open fi

In [8]:
params = dict(
    seed = 123,
    image_dim = (168,168),
    weight_decay = 1e-4,
    epochs = 30,
    batch_size = 4,
    patch_size = 16,
    pool_size = (2,2),
    optimizer = 'Adam',
    l_rate = 0.001,
    val_split = .15,
    use_transfer_learning = False,
    use_data_aug = False,

    l2_reg = .0,
    projection_dim = 16,
    num_heads = 3,
    
    transformer_layers = 4,
    num_classes = 2,
    mlp_head_units = [512,512]
    
    )
new_params = dict(
    num_patches = (params['image_dim'][0] // params['patch_size']) ** 2,
    transformer_units = [
    params['projection_dim'] * 2,
    params['projection_dim']],
    input_shape = (params['image_dim'][0], params['image_dim'][1], 3),

)
params.update(new_params)

In [10]:
import numpy as np
np.random.seed(42)
tf.random.set_seed(42)

def data_pipeline(batch_size, data_dir):
    # Load and preprocess the image data
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        labels="inferred",
        label_mode="categorical",
        class_names=["absent", "present"],
        batch_size=batch_size,
        image_size=params["image_dim"]
    )

    return dataset.shuffle(buffer_size=1000, seed=42)


In [11]:
data_dir = '/kaggle/input/c-nmc-whole'

train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
val_dir = os.path.join(data_dir, 'validation')

train_ds = data_pipeline(params["batch_size"], train_dir)
val_ds = data_pipeline(params["batch_size"], val_dir)
test_ds = data_pipeline(params["batch_size"], test_dir)

Found 13583 files belonging to 2 classes.


2023-07-22 21:45:20.371807: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Found 1746 files belonging to 2 classes.

Found 1698 files belonging to 2 classes.


In [12]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

    
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [13]:
def create_vit_classifier():
    inputs = layers.Input(shape=params['input_shape'])
    patches = Patches(params['patch_size'])(inputs)
    encoded_patches = PatchEncoder(params['num_patches'], params['projection_dim'])(patches)

    for _ in range(params['transformer_layers']):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=params['num_heads'], key_dim=params['projection_dim'], dropout=0.1
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=params['transformer_units'], dropout_rate=0.1)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.expand_dims(representation, axis=-1)
    l2 = layers.MaxPool2D((2, 2), strides=2, padding='same')(representation)
    l3 = layers.Conv2D(32, (3, 3), strides=1, padding='same', activation='relu')(l2)
    l4 = layers.BatchNormalization()(l3)
    l5 = layers.MaxPool2D((2, 2), strides=2, padding='same')(l4)
    l6 = layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')(l5)
    l7 = layers.Dropout(0.1)(l6)
    l8 = layers.BatchNormalization()(l7)
    l9 = layers.MaxPool2D((2, 2), strides=2, padding='same')(l8)
    l13 = layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu')(l9)
    l14 = layers.Dropout(0.2)(l13)
    l15 = layers.BatchNormalization()(l14)
    l16 = layers.MaxPool2D((2, 2), strides=2, padding='same')(l15)
    l17 = layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu')(l16)
    l18 = layers.Dropout(0.2)(l17)
    l19 = layers.BatchNormalization()(l18)
    l20 = layers.MaxPool2D((2, 2), strides=2, padding='same')(l19)
    l21 = layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu')(l20)
    l22 = layers.Dropout(0.2)(l21)
    l23 = layers.BatchNormalization()(l22)
    l24 = layers.MaxPool2D((2, 2), strides=2, padding='same')(l23)
    representation = layers.Flatten()(l24)
    representation = layers.Dropout(0.2)(representation)
    features = mlp(representation, hidden_units=params['mlp_head_units'], dropout_rate=0.2)
    logits = layers.Dense(2)(features)
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [14]:
def run_experiment(model):
    opt = keras.optimizers.Adam(learning_rate=params['l_rate'])
    
    def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

    model.compile(
        optimizer=opt,
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.BinaryAccuracy(name="binary_accuracy"), f1_m, recall_m, precision_m
        ],
    )


    history = model.fit(
        train_ds,
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        validation_data=val_ds
    ) 

    return history, model

model = create_vit_classifier()
history, model = run_experiment(model)

Epoch 1/30

3396/3396 [==============================] - 212s 58ms/step - loss: 0.7176 - binary_accuracy: 0.5290 - f1_m: 0.2007 - recall_m: 0.1625 - precision_m: 0.3079 - val_loss: 0.4890 - val_binary_accuracy: 0.7454 - val_f1_m: 0.7158 - val_recall_m: 0.6693 - val_precision_m: 0.7876

Epoch 2/30

3396/3396 [==============================] - 192s 56ms/step - loss: 0.5214 - binary_accuracy: 0.7155 - f1_m: 0.6647 - recall_m: 0.6077 - precision_m: 0.7700 - val_loss: 0.4287 - val_binary_accuracy: 0.7869 - val_f1_m: 0.7675 - val_recall_m: 0.7294 - val_precision_m: 0.8246

Epoch 3/30

3396/3396 [==============================] - 188s 55ms/step - loss: 0.4853 - binary_accuracy: 0.7519 - f1_m: 0.7226 - recall_m: 0.6777 - precision_m: 0.7953 - val_loss: 0.4156 - val_binary_accuracy: 0.7921 - val_f1_m: 0.7694 - val_recall_m: 0.7288 - val_precision_m: 0.8322

Epoch 4/30

3396/3396 [==============================] - 188s 55ms/step - loss: 0.4654 - binary_accuracy: 0.7651 - f1_m: 0.7387 - recall_m:

In [15]:
model.save('vit/model.h5')